In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import datetime


### After the completion of this notebook 
Milestones achieved (This notebook creates daily notebooks for merging values, after merging, we will use this to make the final merge of all notebooks including instantaneous data, geesebal and the final files generated from here)
1. We closed the energy balance by correcting for LE and H 
2. Removed Netradiaiton outliers (<10) and (>450)
To be achieved
1. Some EBR na's could be filled by taking climatology means of EBR 
(revised the merge notebook after this cleaning)

### Closing daily energy balance 
The daily energy balance requires a 30day moving window to select the median xLE and xH values for the partition of the fluxes
1. Select the names of the columns using the existing code 
2. Make -9999 values nan 
3. Start the 30 day moving window of bowen ratio

In [7]:
# Read ameriflux
# Write the processed hourly files
os.chdir("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Processed_AF_SEBAL\\")
file_list=os.listdir()
file_list
file_name=[]
am_flux=[]
for i in range(len(file_list)):
    print(i)
    am_flux.append(pd.read_csv(file_list[i],parse_dates=["Datetime"]))
    file_name.append(file_list[i].split(".")[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85


In [233]:
names=[]
for i in range(len(am_flux)):
    names.append(am_flux[i]["Name"].iloc[0])

In [8]:
## get average stats of all the stats
## Now the column names are not the same so we extract all cthe variations of the columns names 
## Then we basically take the most popular column name as first choice and if that doesn't work 
# we take the first element of each list. 
le=[] # this iscorrect
h=[] # this also ok
sw_in_list=[]
sw_out_list=[]
TA_list=[]
RH_list=[]
LW_IN_list=[]
LW_OUT_list=[]
NETRAD_list=[]
g=[]
swc=[]
p=[]

# g=[] # this is wrong
for i in range(len(am_flux)):

    le.append(sorted([col for col in am_flux[i].columns if "LE"==col.split("_")[0]],key=len))
#     g.append(sorted([col for col in am_flux[i].columns if "G" in col],key=len))
    h.append(sorted([col for col in am_flux[i].columns if "H"==col.split("_")[0]],key=len))
    g.append(sorted([col for col in am_flux[i].columns if "G"==col.split("_")[0]],key=len))
    sw_in_list.append(sorted([col for col in am_flux[i].columns if "SW_IN" in col],key=len))
    sw_out_list.append(sorted([col for col in am_flux[i].columns if "SW_OUT" in col],key=len))
    TA_list.append(sorted([col for col in am_flux[i].columns if col.startswith("TA_")],key=len))
    RH_list.append(sorted([col for col in am_flux[i].columns if "RH"==col.split("_")[0]],key=len))
    LW_IN_list.append(sorted([col for col in am_flux[i].columns if "LW_IN" in col],key=len))
    LW_OUT_list.append(sorted([col for col in am_flux[i].columns if "LW_OUT" in col],key=len))
    NETRAD_list.append(sorted([col for col in am_flux[i].columns if "NETRAD" in col],key=len))
    swc.append(sorted([col for col in am_flux[i].columns if "SWC"==col.split("_")[0]],key=len))
    p.append(sorted([col for col in am_flux[i].columns if "P"==col.split("_")[0] ],key=len))

In [9]:
print(am_flux[5].columns.to_list())

['Unnamed: 0', 'TIMESTAMP_START', 'TIMESTAMP_END', 'LE', 'H', 'FC', 'USTAR', 'TA', 'WD', 'WS', 'CO2', 'VPD_PI', 'H2O', 'ZL', 'RH', 'SWC_PI_1', 'SWC_PI_2', 'SWC_PI_3', 'SWC_PI_4', 'SWP_PI_1', 'SWP_PI_2', 'SWP_PI_3', 'SWP_PI_4', 'NETRAD', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'G', 'Name', 'Long', 'Lat', 'Elevation', 'State', 'Timezone', 'Veg', 'Clim', 'MAT (°C)', 'MAP (mm)', 'Datetime', 'Date', 'Time', 'doy', 'Albedo', 'dec', 'dr', 'tao', 'Rext', 'Tao_sw_am', 'P_calc', 'es', 'ea', 'W_calc', 'Rso', 'gamma_dust', 'W_prata', 'zenith', 'Mopt', 'asa', 'bsa', 'Tao_sa', 'Tao_ding', 'Tao_sebal_insitu']


In [10]:
netrad_col=[]
le_col=[]
h_col=[]
g_col=[]
swc_col=[]
p_col=[]
# swc_col=[]
## Filter columns based on nan values
for i in range(len(am_flux)):
    a=am_flux[i][NETRAD_list[i]].isna().sum()
    netrad_col.append(a.idxmin())
    a=am_flux[i][le[i]].isna().sum()
    if len(le_col[i])>=3:
        if le_col[i]
    le_col.append(a.idxmin())    
    a=am_flux[i][h[i]].isna().sum()
    h_col.append(a.idxmin())
    a=am_flux[i][g[i]].isna().sum()
    g_col.append(a.idxmin())
    a=am_flux[i][swc[i]].isna().sum()
    swc_col.append(a.idxmin())
    # p[i].remove("P_calc")
    # print(p[i])
    # a=am_flux[i][p[i]].isna().sum()
    # p_col.append(a.idxmin())
        

In [86]:
## Some columns have test QC names in it -- os fixing that
for i in range(len(am_daily)):
    if le_col[i]=="LE_SSITC_TEST_1_1_1" or le_col[i]=="LE_SSITC_TEST":
        print(i)
    # print(le_col[i])
    # print(h_col[i])
    # print(netrad_col[i])
    # print(g_col[i])
print(le[0],le[6],le[27],le[42])
print(h[0],h[6],h[27],h[42])
le_col[0]="LE_1_1_1"
h_col[0]="H_1_1_1"
le_col[6]="LE"
h_col[6]="H"
le_col[27]="LE"
h_col[27]="H"
le_col[42]="LE"
h_col[42]="H"


0
6
27
42


In [87]:
### First we compile the the data into daily data for LE but get stats of the other energy balance variables 
am_daily=[]
for i in range(len(am_flux)):
    print(i)
    var_cleaned=[] # create an empty list 
    for j in range(len(am_flux[i]["Date"].unique())):
        if am_flux[i][am_flux[i]["Date"]==am_flux[i]["Date"].unique()[j]][le_col[i]].isna().sum()<10:
            tmp=am_flux[i][am_flux[i]["Date"]==am_flux[i]["Date"].unique()[j]]
            tmp["LE_na"]=tmp[le_col[i]].isna().sum()
            tmp["Rn_na"]=tmp[netrad_col[i]].isna().sum()
            tmp["h_na"]=tmp[h_col[i]].isna().sum()
            tmp["g_na"]=tmp[g_col[i]].isna().sum()
            tmp["swc_na"]=tmp[swc_col[i]].isna().sum()
            var_cleaned.append(tmp)
            # print(am_flux[i]["Date"].unique()[j])
    if len(var_cleaned)!=0:
        var_new=pd.concat(var_cleaned) # Concatenate the list to a table 
        am_daily.append(var_new.groupby(["Date"]).mean().reset_index())## Group by date to calcualte average 
        am_daily[i]["Name"]=file_name[i]

0


C:\Users\rouhi\AppData\Local\Temp/ipykernel_18720/3139526467.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["LE_na"]=tmp[le_col[i]].isna().sum()
C:\Users\rouhi\AppData\Local\Temp/ipykernel_18720/3139526467.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["Rn_na"]=tmp[netrad_col[i]].isna().sum()
C:\Users\rouhi\AppData\Local\Temp/ipykernel_18720/3139526467.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

KeyboardInterrupt: 

In [251]:
# for i in range(len(am_daily)):
#     am_daily[i].to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Daily_ameriflux_10\\"+am_daily[i]["Name"].iloc[0]+".csv")
os.chdir("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Daily_ameriflux_10\\")
file_list=os.listdir()
am_daily=[]
for i in range(len(file_list)):
    am_daily.append(pd.read_csv(file_list[i],parse_dates=["Date"]))

In [252]:
for i in range(len(am_daily)):
    am_daily[i]["EBR"]=(am_daily[i][le_col[i]]+am_daily[i][h_col[i]])/(am_daily[i][netrad_col[i]]-am_daily[i][g_col[i]])
    

In [253]:
am_daily[0]["EBR"].describe()
am_daily[0][h_col[0]].describe()

count    1964.000000
mean       21.926652
std        28.826005
min      -100.118092
25%         3.019818
50%        18.536768
75%        38.592257
max       122.234382
Name: H_1_1_1, dtype: float64

In [254]:
## Remove outliers
for i in range(len(am_daily)):
    q3, q1 = am_daily[i]["EBR"].describe()[6],am_daily[i]["EBR"].describe()[4]
    # print("Shape of the daily dataframe",am_daily[i].shape[0])
    iqr = q3 - q1
    lb=q1-1.5*iqr
    ub=q3+1.5*iqr
    # print("Lower bound", lb)
    # print("Upper bound", ub)

    am_daily[i]=am_daily[i][(am_daily[i]["EBR"]>=lb) & (am_daily[i]["EBR"]<=ub)]
    print(am_daily[i]["EBR"].mean())
    # print("Shape of the daily dataframe after outlier removal",am_daily[i].shape[0])




0.8046856851270062
0.7812603871029538
0.7502500866790682
0.761082355264575
0.9672939878572424
0.7165643053868227
0.951318340895286
0.41799902073797074
0.6706029387840209
0.7592641777464486
nan
0.6637760675007439
0.7601984113176093
0.7329148995390059
0.9547249114195181
0.8260005715819866
0.7782369394677621
0.7173946649805514
0.747673653806056
1.028384192390921
1.1223784367405558
1.115697669327717
1.089544247332758
0.807032276797785
0.8960658357893804
0.8676258280508036
0.6945868286227077
0.8069700818478432
1.0240644050512893
0.8211979952845098
0.693516880190932
0.720481396090775
0.5801748679717127
0.6609202035448241
0.8956877702525589
0.887886427247886
0.9173388692722887
0.9129852436984672
0.8557403838513548
0.7582500575704673
0.8492816000703658
0.8482375576876628
0.8597561147873967
0.9422027475443775
0.825927920037504
0.9243667463629875
0.9868214827986921
1.0413874544487267
0.8814545347971313
0.8424575726881699
0.9319674152620315
0.9222267238675022
0.9349786595399748
0.8525480287842796

In [255]:

def fill_ebr_gaps(ebr_series,df_no):
    filled_ebr_series = ebr_series.copy().reset_index()
    filled_ebr_series["EBR_cor"]=np.nan
    for i in range(len(ebr_series)):
        # Step 4: Sliding window of +/- 7 days (15 days)
        window_start = ebr_series.Date.iloc[i]-datetime.timedelta(days=7)
        window_end = ebr_series.Date.iloc[i]+datetime.timedelta(days=8)  # +8 to include the current day

        window_values = ebr_series[(ebr_series.Date>=window_start) & (ebr_series.Date<=window_end)]
        print(len(window_values),i)
        # Step 5: If less than +/- 5 days exist, use mean EBR of +/- 5 day sliding window
        if len(window_values)>=11:
            inverse_ebr = 1 / window_values["EBR"].describe().iloc[5]
            # print(inverse_ebr)
            print(inverse_ebr * ebr_series[le_col[df_no]].iloc[i])
            if ((inverse_ebr * ebr_series[le_col[df_no]].iloc[i]) > 850) or ((inverse_ebr * ebr_series[le_col[df_no]].iloc[i]) < -100):
                print("lets go")
                filled_ebr_series.iloc[i,filled_ebr_series.columns.get_loc('EBR_cor')]=np.nan
            else:
                # print("EBR median",window_values["EBR"].describe().iloc[5])
                # print("EBR new",window_values["EBR"].describe().iloc[5] )
                filled_ebr_series.iloc[i,filled_ebr_series.columns.get_loc('EBR_cor')]=window_values["EBR"].describe().iloc[5]
                # filled_ebr_series.at[i,'EBR_cor']=window_values["EBR"].describe().iloc[5]
                # print("Value added", window_values["EBR"].describe().iloc[5])
                # print("value shown", filled_ebr_series.iloc[i,filled_ebr_series.columns.get_loc('EBR_cor')])
        elif len(window_values)<11:
            mean_ebr = window_values.EBR.mean()
            inverse_ebr = 1 / mean_ebr
            # print(inverse_ebr)
            print(inverse_ebr * ebr_series[le_col[df_no]].iloc[i])
            if ((inverse_ebr * ebr_series[le_col[df_no]].iloc[i]) > 850) or ((inverse_ebr * ebr_series[le_col[df_no]].iloc[i]) < -100):
                print("lets go")
                filled_ebr_series.iloc[i,filled_ebr_series.columns.get_loc('EBR_cor')]=np.nan
            else: 
                filled_ebr_series.iloc[i,filled_ebr_series.columns.get_loc('EBR_cor')]=mean_ebr
        else: ## Probably need to add climatology in the future 
            filled_ebr_series.iloc[i,filled_ebr_series.columns.get_loc('EBR_cor')]=np.nan

                                    
        # if len(window_values) < 11 or filled_ebr_series["EBR_cor"][i]=-999:
        #     mean_ebr = ebr_series[max(0, i - 5):min(len(ebr_series), i + 6)].mean()
        #     window_values = np.append(window_values, [mean_ebr] * (15 - len(window_values)))

        # # Step 6: Fill remaining gaps with mean from a +/- 5 day sliding window climatology
        # if np.isnan(window_values).all():
        #     climatology_mean = ebr_series[max(0, i - 5):min(len(ebr_series), i + 6)].mean()
        #     window_values = np.append(window_values, [climatology_mean] * (15 - len(window_values)))

        # # Calculate inverse EBR
        # inverse_ebr = 1 / window_values

        # # Check conditions for extreme EBR values
        # if (
        #     (abs(inverse_ebr) > 2).any()
        #     or ((inverse_ebr * measured_le) > 850).any()
        #     or ((inverse_ebr * measured_le) < -100).any()
        # ):
        #     filled_ebr_series.iloc[i] = np.nan
    print("value shown outside", filled_ebr_series['EBR_cor'])

    return filled_ebr_series
am_ebr=[]
for i in range(len(am_daily)):
    am_ebr.append(fill_ebr_gaps(am_daily[i],i))

3 0
7.577784277143519
5 1
19.50001386889858
8 2
10.36969455429309
8 3
11.353256083095625
10 4
13.878093210756727
11 5
9.851665186964587
12 6
16.306225742561445
12 7
20.59623582724694
12 8
24.621415745055682
13 9
22.862226234461865
14 10
5.885446801786997
13 11
18.72316197419247
13 12
16.014371668130476
12 13
22.553425867348544
11 14
27.73048048181724
12 15
26.50120133455637
11 16
31.251369424542155
11 17
39.24604202088083
11 18
11.759619739758493
11 19
40.437805441011726
10 20
49.19813318352856
10 21
31.697420350889516
11 22
38.80441364729442
10 23
45.08074330370638
11 24
22.101858176010122
11 25
27.994977271353015
12 26
19.970620427114838
12 27
23.90217577878258
11 28
37.12264953858877
12 29
35.33204162160866
11 30
17.257891964418146
11 31
19.85297341235661
10 32
9.167106071339218
9 33
22.156500225291474
8 34
31.986366892344435
8 35
37.62631317668261
7 36
34.729205811227764
3 37
73.38498359671418
5 38
44.10618769368722
4 39
83.31342746554199
5 40
49.75213196089224
6 41
64.377424874755

In [256]:
## Do not have that many nans 
for i in range(len(am_daily)):
    print(am_ebr[i]["EBR_cor"].isna().sum(),am_ebr[i].shape[0])
### Tests of the data to see EBR-new is being calculted correctly
## Check if the 
for i in range(len(am_ebr)):
    print(am_ebr[i][["EBR_cor","EBR"]].describe())
## The disribution is also similar 

0 1517
0 1974
0 1716
0 782
0 1724
0 26
0 1605
18 711
1 329
0 860
0 0
0 1984
0 1002
0 1021
0 1573
0 1619
0 1543
0 1363
0 1089
0 891
0 553
0 867
0 602
0 1697
0 1751
0 1472
0 2248
0 1470
0 1277
0 1069
0 219
0 1237
0 1279
0 1136
0 1137
0 1245
0 2225
0 2192
0 49
0 2429
0 949
0 312
0 349
0 325
0 2330
0 2357
0 2051
0 113
0 448
0 360
0 257
0 259
0 236
0 508
0 153
0 225
0 167
0 377
0 260
0 190
0 361
0 496
0 44
0 141
0 184
0 426
0 134
34 186
0 380
0 207
0 106
0 127
0 297
0 29
0 199
0 21
0 295
0 386
0 318
0 20
0 471
0 446
0 442
0 226
0 93
0 91
           EBR_cor          EBR
count  1517.000000  1517.000000
mean      0.800650     0.804686
std       0.093455     0.158631
min       0.479644     0.399610
25%       0.768743     0.721100
50%       0.811883     0.808987
75%       0.849970     0.888683
max       1.064650     1.226422
           EBR_cor          EBR
count  1974.000000  1974.000000
mean      0.778530     0.781260
std       0.064423     0.098030
min       0.581870     0.520422
25%       0.7

In [257]:
### Now close the EB and remove NET radiation errors 
## we consider daily net rad to be erroneous if it;'s negative (snow) or greater than 400 (subjective)
def close_eb(df,df_num):
    df["Residual_unclosed"]=df[netrad_col[df_num]]-df[g_col[df_num]]-df[le_col[df_num]]-df[h_col[df_num]]
    df["LE_closed"]=df[le_col[df_num]]/df["EBR_cor"]
    df["H_closed"]=df[h_col[df_num]]/df["EBR_cor"]
    df["Residual_closed"]=df[netrad_col[df_num]]-df[g_col[df_num]]-df["LE_closed"]-df["H_closed"]
    df["LE_unclosed"]=df[le_col[df_num]]
    df["H_unclosed"]=df[h_col[df_num]]
    df["G_unclosed"]=df[g_col[df_num]]
    df["Rn_unclosed"]=df[netrad_col[df_num]]


    df=df[(df[netrad_col[df_num]]>=10) & (df[netrad_col[df_num]]<=400)]
    return df
for i in range(len(am_ebr)):
    am_ebr[i]=close_eb(am_ebr[i],i)

In [258]:
### Let's take a look if closing reduces the residual 
## Residual looks good. It's closing the EB 
for i in range(len(am_ebr)):
    # print(am_ebr[i][["Residual_unclosed","Residual_closed","LE_closed",le_col[i],netrad_col[i]]].describe())
    # if am_ebr[i].shape[0]!=np.nan:
    # print(am_ebr[i][am_ebr[i][netrad_col[i]]>=10].shape[0],am_ebr[i][["Lat","Long"]])
    # print(am_ebr[i][am_ebr[i][netrad_col[i]]>=10].shape[0])
    print(am_ebr[i].shape[0],)

###Let's take a look the netradiation errors 

1498
1952
1705
780
1671
26
1597
693
320
789
0
1978
1000
1021
1531
1592
1498
1306
1070
886
549
867
595
1646
1685
1413
2233
1469
1206
987
179
1077
1051
924
1030
1192
2213
2183
49
2368
937
306
349
312
2302
2351
2043
106
447
350
254
259
227
444
150
225
167
367
259
188
340
469
43
134
178
389
132
102
343
207
106
124
289
29
196
21
294
367
317
20
456
425
416
203
86
91


In [260]:
for i in range(len(am_ebr)):
    am_ebr[i]["Name"]=names[i]

In [261]:
## Print and save the csv files
for i in range(len(am_ebr)):
    am_ebr[i].to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Daily_Closed\\"+names[i]+".csv")
